# Part 1: Motivation
Let's say we're building a local editor that allows you to load an AIConfig
from a local file and then run methods on it.

In the (simplified) code below, we do just that.

In [16]:
import json
from typing import Any


def read_json_from_file(path: str) -> dict[str, Any]:
    with open(path, "r") as f:
        return json.loads(f.read())
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")


start_app("cookbooks/Getting-Started/travel.aiconfig.json")

Loaded AIConfig: NYC Trip Planner




## Cool, LGTM, ship it!

# A few hours later...

# Issue #9000 Editor crashes on new file path
### opened 2 hours ago by lastmile-biggest-fan

Dear LastMile team,
I really like the editor, but when I give it a new file path, it crashes!
I was hoping it would create a new AIConfig for me and write it to the file...

# OK, what happened?

In [17]:
start_app("i-dont-exist-yet-please-create-me.json")

FileNotFoundError: [Errno 2] No such file or directory: 'i-dont-exist-yet-please-create-me.json'

# Oops

Ok, let's diagnose the problem here. We forgot to handle the case where the path doesn't exist.

That's understandable. As programmers, we don't always write perfect code.
Sometimes it's helpful to bring new tools into the workflow to prevent this kind of problem in the future.


Hmm, ok. Wouldn't it be nice if we had a static analyzer that could have caught this problem immediately? That way we could have fixed it before the initial PR was merged.

Let's analyze some tools.

## V2: Optional

First, let's fix the root cause and catch exceptions. Now, what do we do in the `except` block? 

Well, we can reraise, but that brings us right back to the previous case and doesn't achieve anything helpful. 

Instead, notice what happens if we return None and type hint the function accordingly (Optional[...]).

In [6]:
from typing import Any, Optional


def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    try:
        with open(path, "r") as f:
            return json.loads(f.read())
    except Exception as e:
        return None
    

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    print(f"Loaded AIConfig: {aiconfig['name']}\n")

print("""
[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None
""")


[Pyright] Object of type "None" is not subscriptable
PylancereportOptionalSubscript
(variable) aiconfig: dict[str, Any] | None



# Aha!

Now, Pyright immediately tells us that `None` is a possibility, and we have to handle this case. Let's do that.


In [31]:
from typing import Optional
from aiconfig.Config import AIConfigRuntime



def read_json_from_file(path: str) -> Optional[dict[str, Any]]:
    try:
        with open(path, "r") as f:
            return json.loads(f.read())
    except Exception:
        return None

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    aiconfig = read_json_from_file(path)
    if aiconfig is None:
        print(f"Could not load AIConfig from path: {path}. Creating and saving.")
        aiconfig = json.dumps(AIConfigRuntime.create())
        # [save the aiconfig to the path]        
        print(f"Loaded and saved new AIConfig\n")
    else:
        print(f"Loaded AIConfig: {aiconfig}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Loaded AIConfig: 



Ok, cool, much better. But wait, it would be nice to retain some information about what went wrong. My `None` value doesn't tell me anything about why the AIConfig couldn't be loaded. Does the file not exist? Was it a permission problem, networked filesystem problem? etc.

# V3: Result

The result library (https://github.com/rustedpy/result) provides a neat type
called `Result`, which is a bit like Optional. It's parametrized by the value type just like optional, but also by a second type for the error case.

We can use it like optional, but store an arbitrary value with information about what went wrong.

In [52]:
from aiconfig.Config import AIConfigRuntime
from result import Result, Ok, Err
from typing import Any

from json import JSONDecodeError


def read_json_from_file(path: str) -> Result[dict[str, Any], str]:
    """Use `str` in the error case to contain a helpful error message."""
    try:
        with open(path, "r") as f:
            return Ok(json.loads(f.read()))
    except FileNotFoundError:
        return Err(f"File not found at path: {path}")
    except OSError as e:
        return Err(f"Could not read file at path: {path}: {e}")
    except JSONDecodeError as e:
        return Err(f"Could not parse JSON at path: {path}: {e}")

def start_app(path: str):
    """Load an AIConfig from a local path and do something with it."""
    file_contents = read_json_from_file(path)
    match file_contents:
        case Ok(aiconfig_ok):
            print(f"Loaded AIConfig: {aiconfig_ok['name']}\n")
        case Err(e):
            print(f"Could not load AIConfig from path: {path} ({e}). Creating and saving.")
            aiconfig = AIConfigRuntime.create().model_dump(exclude="callback_manager")
            # [Save to file path]
            # aiconfig.save(path)
            print(f"Created and saved new AIConfig: {aiconfig['name']}\n")

start_app("cookbooks/Getting-Started/travel.aiconfig.json")
start_app("i-dont-exist-yet-please-create-me.json")

Loaded AIConfig: NYC Trip Planner

Could not load AIConfig from path: i-dont-exist-yet-please-create-me.json (File not found at path: i-dont-exist-yet-please-create-me.json). Creating and saving.
Created and saved new AIConfig: 



There are several nice things about this pattern:
* If you fail to check for the error case, you get static errors similar to the `None` Optional case
* You also get specific, useful error information unlike Optional
* Structural pattern matching: When matching the cases, you can elegantly and safely unbox the data inside the result.
* Because of pyright's ability to check for exhaustive pattern matching, it will yell at you if you don't handle the Err case. Try it! Comment out the Err case.

# Part 2: Composition

Cool, so we have a very basic example of better error handling. What about a more realistic level of complexity involving a sequence of chained operations? Consider this variant of the previous app example:

In [5]:
from dataclasses import dataclass
import json
from typing import NewType

JSON = NewType("JSON", dict[str, Any])

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

def parse_json(json_str: str) -> JSON:
    return json.loads(json_str)


def read_json_from_file(path: str) -> JSON:
    with open(path, "r") as f:
        return json.loads(f.read())

def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""
    contents = read_file(path)
    aiconfig = parse_json(contents)
    prompts = get_prompt_text_list(aiconfig)
    return HTTPResponse(200, JSON({"prompts": prompts}))



endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

KeyError: 'text'

We should not let this propagate to the server framework. It will get caught and shown to the user as an unhelpful stack trace. Even if the frontend handles generic errors, it cannot do anything useful. Instead, we should return specific HTTP errors.

In [7]:
from dataclasses import dataclass
import json
from typing import NewType

JSON = NewType("JSON", dict[str, Any])

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

def parse_json(json_str: str) -> JSON:
    return json.loads(json_str)


def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))

def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    try:
        contents = read_file(path)
        aiconfig = parse_json(contents)
        prompts = get_prompt_text_list(aiconfig)
        response = prompt_list_to_http_response(prompts)
        return response
    except FileNotFoundError:
        return HTTPResponse(404, JSON({"error": "File not found"}))
    except OSError as e:
        return HTTPResponse(500, JSON({"error": f"Could not read file: {e}"}))
    except json.JSONDecodeError as e:
        return HTTPResponse(555, JSON({"error": f"Could not parse JSON: {e}"}))
    except KeyError as e:
        return HTTPResponse(567, JSON({"error": f"Could not find key: {e}"}))


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'error': "Could not find key: 'text'"})

This is better, but we can't reuse any of our helper functions! Every endpoint will have to repeat this error handling. OK, new version:


In [11]:
from dataclasses import dataclass
import json
from typing import NewType
from result import Result, Ok, Err

JSON = NewType("JSON", dict[str, Any])

@dataclass
class InternalFailure:
    message: str
    category: str
    exception: Exception

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON

def read_file(path: str) -> Result[str, InternalFailure]:
    try:
        with open(path, "r") as f:
            return Ok(f.read())
    except Exception as e:
        return Err(InternalFailure(
            message=f"Could not read file at path: {path}",
            category="OSError",
            exception=e,
        ))
    

def parse_json(json_str: str) -> Result[JSON, InternalFailure]:
    try: 
        return Ok(json.loads(json_str))
    except json.JSONDecodeError as e:
        return Err(InternalFailure(
            message="Could not parse JSON",
            category="JSONDecodeError",
            exception=e,
        ))



def get_prompt_text_list(aiconfig: JSON) -> Result[list[str], InternalFailure]:
    try:
        return Ok([prompt["text"] for prompt in aiconfig["prompts"]])
    except KeyError as e:
        return Err(InternalFailure(
            message="Could not find key",
            category="KeyError",
            exception=e,
        ))

def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    if failure.category == "OSError":
        return HTTPResponse(500, JSON({"error": f"Could not read file: {failure.exception}"}))
    elif failure.category == "JSONDecodeError":
        return HTTPResponse(555, JSON({"error": f"Could not parse JSON: {failure.exception}"}))
    elif failure.category == "KeyError":
        return HTTPResponse(567, JSON({"error": f"Could not find key: {failure.exception}"}))
    else:
        return HTTPResponse(500, JSON({"error": f"Internal failure: {failure}"}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    file_contents = read_file(path)
    match file_contents:
        case Ok(file_contents_ok):
            parsed_json = parse_json(file_contents_ok)
            match parsed_json:
                case Ok(parsed_json_ok):
                    prompts = get_prompt_text_list(parsed_json_ok)
                    match prompts:
                        case Ok(prompts_ok):
                            return prompt_list_to_http_response(prompts_ok)
                        case Err(e):
                            return internal_failure_to_http_response(e)
                case Err(e):
                    return internal_failure_to_http_response(e)
        case Err(e):
            return internal_failure_to_http_response(e)


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'error': "Could not find key: InternalFailure(message='Could not find key', category='KeyError', exception=KeyError('text'))"})

Ok, but Now we still have to do this annoying nested error checking! Luckily, Result has a really nice way to do this.

In [12]:
from dataclasses import dataclass
import json
from typing import NewType
from result import Result, Ok, Err

JSON = NewType("JSON", dict[str, Any])

@dataclass
class InternalFailure:
    message: str
    category: str
    exception: Exception

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON
    

def read_file(path: str) -> Result[str, InternalFailure]:
    try:
        with open(path, "r") as f:
            return Ok(f.read())
    except Exception as e:
        return Err(InternalFailure(
            message=f"Could not read file at path: {path}",
            category="OSError",
            exception=e,
        ))
    

def parse_json(json_str: str) -> Result[JSON, InternalFailure]:
    try: 
        return Ok(json.loads(json_str))
    except json.JSONDecodeError as e:
        return Err(InternalFailure(
            message="Could not parse JSON",
            category="JSONDecodeError",
            exception=e,
        ))



def get_prompt_text_list(aiconfig: JSON) -> Result[list[str], InternalFailure]:
    try:
        return Ok([prompt["text"] for prompt in aiconfig["prompts"]])
    except KeyError as e:
        return Err(InternalFailure(
            message="Could not find key",
            category="KeyError",
            exception=e,
        ))

def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    if failure.category == "OSError":
        return HTTPResponse(500, JSON({"error": f"Could not read file: {failure.exception}"}))
    elif failure.category == "JSONDecodeError":
        return HTTPResponse(555, JSON({"error": f"Could not parse JSON: {failure.exception}"}))
    elif failure.category == "KeyError":
        return HTTPResponse(567, JSON({"error": f"Could not find key: {failure.exception}"}))
    else:
        return HTTPResponse(500, JSON({"error": f"Internal failure: {failure}"}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    response = (
        read_file(path)
        .and_then(parse_json)
        .and_then(get_prompt_text_list)
        .map(prompt_list_to_http_response)
        .unwrap_or_else(internal_failure_to_http_response)
    )
    return response


endpoint("cookbooks/Getting-Started/travel.aiconfig.json")

HTTPResponse(status_code=567, body={'error': "Could not find key: 'text'"})

Neat. To recap, this code now has the following very nice properties:
* Errors are statically checked, eliminating a class of bugs
* Modular and highly reusable
* Concise (especially the endpoint)

# Part 3: Advanced topics
* decorators and other higher-order functions
* partial application
* do-notation for complex composition

In [ ]:
from functools import partial
from typing import Callable, TypeVar, ParamSpec

R = TypeVar('R')
P = ParamSpec('P')
E = TypeVar('E')

def parametrized(
    decorator: Callable[[Callable[P, R], Callable[..., Result[R, E]]], Callable[P, Result[R, E]]]
) -> Callable[[Callable[..., Result[R, E]]], Callable[[Callable[P, R]], Callable[P, Result[R, E]]]]:
    def layer(exception_handler: Callable[..., Result[R, E]]) -> Callable[[Callable[P, R]], Callable[P, Result[R, E]]]:
        def wrapper(func: Callable[P, R]) -> Callable[P, Result[R, E]]:
            return decorator(func, exception_handler)
        return wrapper
    return layer

@parametrized
def exception_handled(func: Callable[P, R], exception_handler: Callable[..., Result[R, E]]) -> Callable[P, Result[R, E]]:
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> Result[R, E]:
        try:
            return Ok(func(*args, **kwargs))
        except Exception as e: # catch all exceptions and delegate to handler
            return exception_handler(e)
    return wrapper

from dataclasses import dataclass
import json
from typing import NewType
from result import Result, Ok, Err

JSON = NewType("JSON", dict[str, Any])

@dataclass
class InternalFailure:
    message: str
    category: str
    exception: Exception

@dataclass
class HTTPResponse:
    status_code: int
    body: JSON
    

def _handle_exception_with_helper(exception: Exception, message: str, category: str) -> Err[InternalFailure]:
    return Err(InternalFailure(message, category, exception))

def handle_exception_with(message: str, category: str) -> Callable[[Exception], Err[InternalFailure]]:
    return partial(_handle_exception_with_helper, message=message, category=category)

@exception_handled(handle_exception_with("Could not read file", "OSError"))
def read_file(path: str) -> str:
    with open(path, "r") as f:
        return f.read()
    

@exception_handled(handle_exception_with("Could not parse JSON", "JSONDecodeError"))
def parse_json(json_str: str) -> JSON:
    return JSON(json.loads(json_str))


@exception_handled(handle_exception_with("Could not find key", "KeyError"))
def get_prompt_text_list(aiconfig: JSON) -> list[str]:
    return [prompt["text"] for prompt in aiconfig["prompts"]]


def prompt_list_to_http_response(prompts: list[str]) -> HTTPResponse:
    return HTTPResponse(200, JSON({"prompts": prompts}))


def internal_failure_to_http_response(failure: InternalFailure) -> HTTPResponse:
    if failure.category == "OSError":
        return HTTPResponse(500, JSON({"error": f"Could not read file: {failure.exception}"}))
    elif failure.category == "JSONDecodeError":
        return HTTPResponse(555, JSON({"error": f"Could not parse JSON: {failure.exception}"}))
    elif failure.category == "KeyError":
        return HTTPResponse(567, JSON({"error": f"Could not find key: {failure.exception}"}))
    else:
        return HTTPResponse(500, JSON({"error": f"Internal failure: {failure}"}))


def endpoint(path: str):
    """Load an AIConfig from a local path and do something with it."""

    response = (
        read_file(path)
        .and_then(parse_json)
        .and_then(get_prompt_text_list)
        .map(prompt_list_to_http_response)
        .unwrap_or_else(internal_failure_to_http_response)
    )
    return response

